<img src="https://i.imgur.com/oiG2jZl.png">
<center><h1>🧭Indoor Location and Navigation🧭</h1></center>

# 1. Introduction
> 📌**Goal**: リアルタイムセンサーによるスマートフォン📱の室内位置の予測🎯.

> また、今回のコンペで利用できる**GitHub Repository**を使って、カスタム関数を**ノートにコピーペーストせずに**呼び出す方法も学びます。

### Libraries📚

In [ ]:
# CPU libraries
import os
import json
import glob # パターンに一致するすべてのパス名を見つける
import cv2  # OpenCV(画像の読み書き・リサイズ・反転などの加工)
import numpy as np
import pandas as pd
import seaborn as sns # データ可視化(中でmatplotlibを使ってる)
import matplotlib     # データ可視化
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps # 画像処理(参考：https://note.nkmk.me/python-pillow-basic/)
from skimage import io          # 画像処理(参考：https://qiita.com/supersaiakujin/items/fc54116df9ca6958a68d)
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm           # プログレスバー
from dataclasses import dataclass  # __init__() のような特殊なクラスを生成
from math import floor, ceil

mycolors = ["#797D62", "#9B9B7A", "#D9AE94", "#FFCB69", "#D08C60", "#997B66"]

In [ ]:
def show_values_on_bars(axs, h_v="v", space=0.4):
    '''seabornの棒グラフの最後に値をプロットします。
    axs: プロットの軸
    h_v: バープロットが垂直／水平かどうか'''
    
    def _show_on_single_plot(ax):
        if h_v == "v":  # 垂直の場合
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height()
                value = int(p.get_height())
                ax.text(_x, _y, format(value, ','), ha="center") 
        elif h_v == "h": # 水平の場合
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height()
                value = int(p.get_width())
                ax.text(_x, _y, format(value, ','), ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

### W&Bのセットアップ
* アカウントの作成 https://wandb.ai (it's free)
* menu -> Add-ons -> Secrets -> Add a new Secret -> label=wandb-key value=wandb API key
* Install `wandb`
* プロジェクトの「パーソナル・キー」を入力してください。 ( 私の場合は機密なので、秘密にしておきます。 :) )

In [ ]:
import wandb
os.environ["WANDB_SILENT"] = "true"

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
personal_key_for_api = user_secrets.get_secret("wandb-key")

In [ ]:
# wandbにログイン
! wandb login $personal_key_for_api

In [ ]:
# 新規プロジェクトの初期化
### project - overajjプロジェクトの名前（GitHubのリポジトリの名前と同じ)
### name/experiment - runの名前（1つのプロジェクトで複数のrunを使用)
run = wandb.init(project="indoor-location-kaggle", name="data-understanding")

> We're all set!

# 2. Data Understanding

* `train` - *site*と*floor*で構成された学習用パスファイル。各パスファイルには、1つのフロアの1つのパスのデータが含まれる。
* `test` - テスト用のパスファイルで、*site*と*floor*で構成されています。各パスファイルには、1つのフロアの1つのパスのデータが含まれていますが、**waypoint (x, y)データ**は含まれていません。
* `metadata` - フロアメタデータフォルダ（サイトとフロアごとに整理されており、フロアごとに以下を含む:
    * `floor_image.png`
    * `floor_info.json`
    * `geojson_map.json`
    
<img src="https://i.imgur.com/EE98923.png" width=500>

> 📌**Goal**: このコンペの課題は、与えられたsite-pathファイルに対して、`sample_submission.csv`ファイルで与えられた**timestamp**で、**predict the floor**と**waypoint locations**を予測することです。

> ❗**Note on data quality**❗: 学習用ファイルの中には、最後の改行文字がない行があり、次の行に進んでしまうことがあります。この問題をどのように処理するかは、ユーザー次第です。この問題は、テストデータでは見られません。

In [ ]:
# ファイル数を確認
train_paths = glob.glob('../input/indoor-location-navigation/train/*/*/*')
test_paths = glob.glob('../input/indoor-location-navigation/test/*')
sites = glob.glob('../input/indoor-location-navigation/metadata/*')

print("No. Files in Train: {:,}".format(len(train_paths)), "\n" +
      "No. Files in Test: {:,}".format(len(test_paths)), "\n" +
      "Total Sites (metadata): {:,}".format(len(sites)))

In [ ]:
# 実験的にW&Bの"data-understanding"にログを残す。W&Bのサイト参照
# 参考：https://wandb.ai/honda/indoor-location-kaggle/runs/36e88gwf?workspace=user-honda
wandb.log({'No. Files in Train': len(train_paths), 
           'No. Files in Test:' : len(test_paths),
           'Total Sites (metadata)' : len(sites)})

## データ読み込み

> ❗**Note**: 提示されたデータは`.txt`形式で、多くの情報が含まれています。たった1つの`.txt`ファイル（つまり1つのパス）に、126k行以上の情報がありました。1つのファイルごとに情報は異なるので、すべてを構造化しようとすると面倒なことになります。

In [ ]:
# 1ファイルをのぞいてみる
base = '../input/indoor-location-navigation'
path = f'{base}/train/5cd56b5ae2acfd2d33b58549/5F/5d06134c4a19c000086c4324.txt'

with open(path) as p:
    lines = p.readlines()

print("No. Lines in 1 example: {:,}". format(len(lines)), "\n" +
      "Example (5 lines): ", lines[0:5])

# 2. KaggleでのGitHub repoの使い方?🔗

> 📌**Goal**: このコンペには、[GitHub repo](https://github.com/location-competition/indoor-location-competition-20)が用意されています。io_f`ファイルの`read_data_file`関数を使って情報を読み込めばいいのです（こちら側での追加の苦労はありません＋コピーペーストでコードが乱雑になることもありません :)）

#### *🙏🏻この素晴らしいトリックを教えてくれた[Laura](https://www.kaggle.com/allunia)に感謝します。🙏🏻*

**Steps**:
* 🦶🏻 - このリンクからrepoをダウンロードする(https://github.com/location-competition/indoor-location-competition-20)
* 🦶🏻 - パッケージをKaggle環境にコピーします。 (`!cp -r path/* ./`)
* 🦶🏻 - importして好きなように使う

In [ ]:
!cp -r ../input/indoor-locationnavigation-2021/indoor-location-competition-20-master/indoor-location-competition-20-master/* ./

In [ ]:
# リポジトリからカスタム関数をインポート
from io_f import read_data_file

# 1つのランダムなファイルで読む
sample_file = read_data_file(path)

# 各変数の情報にアクセスできます:
print("~~~ Example ~~~")
# acce:TYPE_ACCELEROMETER:加速度計
# acacce_uncalice: TYPE_ACCELEROMETER_UNCALIBRATED:加速度計(補正前？)
# ahrs:TYPE_ROTATION_VECTOR：回転ベクトル
# gyro:TYPE_GYROSCOPE：ジャイロスコープ(物体の角度（姿勢）や角速度あるいは角加速度を検出する計測器ないし装置)
# ibeacn:TYPE_BEACON：ビーコン(Bluetooth low energyのブロードキャスト通信を利用したiOSの近接通知機能)
# magn:TYPE_MAGNETIC_FIELD：磁力計
# magn_uncali:TYPE_MAGNETIC_FIELD_UNCALIBRATED：磁力計（補正前？）
# waypoint:TYPE_WAYPOINT：歩いた場所
# wifi:TYPE_WIFI： wifi
print("acce: {}".format(sample_file.acce.shape), "\n" +
      "acacce_uncalice: {}".format(sample_file.acce_uncali.shape), "\n" +
      "ahrs: {}".format(sample_file.ahrs.shape), "\n" +
      "gyro: {}".format(sample_file.gyro.shape), "\n" +
      "gyro_uncali: {}".format(sample_file.gyro_uncali.shape), "\n" +
      "ibeacon: {}".format(sample_file.ibeacon.shape), "\n" +
      "magn: {}".format(sample_file.magn.shape), "\n" +
      "magn_uncali: {}".format(sample_file.magn_uncali.shape), "\n" +
      "waypoint: {}".format(sample_file.waypoint.shape), "\n" +
      "wifi: {}".format(sample_file.wifi.shape))

よし、今から楽しもう！

## Sites🏢

> **site**はエンコードされており、データが抽出された**建物**を表しています。

In [ ]:
def show_site_png(site):
    '''この機能は、メタデータに含まれる.png画像を可視化して出力します。
    sites: 1つのサイト（または建物）に対応するコード'''
    
    base = '../input/indoor-location-navigation'
    site_path = f"{base}/metadata/{site}/*/floor_image.png"
    floor_paths = glob.glob(site_path)
    n = len(floor_paths)

    # カスタムの行と列の数を作成 1行につき3つまで表示できるようにする。
    ncols = [ceil(n / 3) if n > 4 else 4][0]
    nrows = [ceil(n / ncols) if n > 4 else 1][0]

    plt.figure(figsize=(16, 10))
    plt.suptitle(f"Site no. '{site}'", fontsize=18)

    # 各階のプロットイメージ
    for k, floor in enumerate(floor_paths):
        plt.subplot(nrows, ncols, k+1)

        image = Image.open(floor)
        image = ImageOps.expand(image, border=15, fill=mycolors[5])

        plt.imshow(image)
        plt.axis("off")
        title = floor.split("/")[5]
        plt.title(title, fontsize=15)
        
        # W&Bの"data-understanding"に 実験的にログ出力する
        wandb.log({"Site Floors Example": plt})

In [ ]:
# 1つの例を見てみよう
# site = '5cd56b64e2acfd2d33b59246'
show_site_png(site='5cd56b64e2acfd2d33b592b3')

## いくつのフロアがある? 🛩

> ❗**Note**: この変数は、予測が必要なターゲット変数の1つであるため、非常に重要です。

In [ ]:
all_floors = glob.glob("../input/indoor-location-navigation/metadata/*/*")
floor_no = []

# フロア番号のみを抽出
for floor in all_floors:
    no = floor.split("/")[5]
    floor_no.append(no)
    
floor_no = pd.DataFrame(floor_no, columns=["No"])
floor_no = floor_no["No"].value_counts().reset_index()
floor_no = floor_no.sort_values("No", ascending=False)

# ~~~~
# フロアごとに数を表示
# ~~~~
plt.figure(figsize=(16, 12))
ax = sns.barplot(data=floor_no, x="No", y="index", palette="Greens_r",
                 saturation=0.4)
show_values_on_bars(ax, h_v="h", space=0.4)
ax.set_title("Frequency of Floors", size = 26, color = mycolors[0], weight='bold')
ax.set_xlabel("")
ax.set_ylabel("Floor No.", size = 18, color = mycolors[0], weight='bold')
plt.xticks([])
plt.yticks(fontsize=11)
sns.despine(left=True, bottom=True);

In [ ]:
# W&Bの"data-understanding"に実験的にログ出力する。
### 現在、seabornのダイレクトログはサポートされていません。
### カスタム棒グラフを作成します。
data = [[index, no] for (index, no) in zip(floor_no["index"], floor_no["No"])]
table = wandb.Table(data=data, columns=["index", "no"])
wandb.log({"Frequency of Floors" : wandb.plot.bar(table, "index", "no", title="Frequency of Floors")})

> W&Bのダッシュボードではこのように表示されます:
<img src="https://i.imgur.com/IpxLc7t.png" width=700>

## Waypoint

> ❗**Note**: この変数は非常に重要で、我々が予測しなければならないもう一つのターゲット変数です。

> 📌**Waypoint**: 下の図は、1人の人間が1つのサイトの5階を通過する際の*経路*、つまり軌跡を示しています。かなり歩いて往復しています。また、この軌跡の**開始点**と**終了点**を見ることができます。

In [ ]:
# GitHub functions
from visualize_f import visualize_trajectory, visualize_heatmap

path = f'{base}/train/5cd56b5ae2acfd2d33b58549/5F/5d06134c4a19c000086c4324.txt'

# サンプルの読み込み
example = read_data_file(path)

# ~~~~~~~~~

# タイムスタンプとx,y座標の値を返す
trajectory = example.waypoint
# タイムスタンプの削除（必要なのは座標のみ)
trajectory = trajectory[:, 1:3]

# 例題に沿ったfloor_planを用意する
# ../input/indoor-location-navigation/metadata/5a0546857ecc773753327266/B1/xxxxx.xxx というようなディレクトリ構成
site = path.split("/")[4]
floorNo = path.split("/")[5]
floor_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_image.png'

# width_meter & height_meterを準備する
### (taken from the .json file)
json_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_info.json'
with open(json_plan_filename) as json_file:
    json_data = json.load(json_file)
    
# {"map_info": {"height": 204.53342955266643, "width": 270.34143433711995}} こんなデータ
width_meter = json_data["map_info"]["width"]
height_meter = json_data["map_info"]["height"]

# Title
title = "Example of Waypoint"

# ~~~~~~~~~

# Finally, let's plot
visualize_trajectory(trajectory = trajectory,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                     title = title,
                     g_size=755,
                     point_color='#76C1A0',
                     start_color='#007B51',
                     end_color='#9B0000')

## 磁力の強さ 🧲

> 📌**Magnetic Strength**: 建物内のどの地点も、**固有の磁力**を受けています。床や壁、あるいは部屋の周囲にある物体は、3次元空間と**磁気の大きさ**の4次元地図を作成します。この空間内の任意の点における磁気の大きさは、その点におけるx、y、zの磁気ベクトルを読み取ることで測定できる。

> ❗**Note**: 携帯電話は、ユーザーが移動すると（携帯電話が回転すると）、磁界の変動を検出します。下の例では、waypointの最初の方では磁場が強く、床の左側では磁場が弱くなっています。

#### 📐mu tesla (1×104 G) - 磁気誘導の派生単位📐

In [ ]:
# GitHub functions
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength

# 磁力の値を取得する。
mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position([path])
magnetic_strength = extract_magnetic_strength(mwi_datas)

heat_positions = np.array(list(magnetic_strength.keys()))
heat_values = np.array(list(magnetic_strength.values()))

# ヒートマップの可視化
visualize_heatmap(heat_positions, 
                  heat_values, 
                  floor_plan_filename,
                  width_meter, 
                  height_meter, 
                  colorbar_title='mu tesla', 
                  title='Magnetic Strength',
                  g_size=755,
                  colorscale='temps')

## WiFi 📶

> 📌**WiFi Access Points**: 敷地内のフロアには、WiFiアクセスポイントが非常に多く存在します。そのため、エリアによって信号やその強さが大きく異なります。
> ❗**Note**: 下の例では、ルート上にこのようなアクセスポイントがたくさんあることがわかります。

In [ ]:
# GitHub Libraries
from main import extract_wifi_rssi, extract_wifi_count

# Get WiFi data
wifi_rssi = extract_wifi_rssi(mwi_datas)
print(f'This floor has {len(wifi_rssi.keys())} wifi aps (access points).')

wifi_counts = extract_wifi_count(mwi_datas)
heat_positions = np.array(list(wifi_counts.keys()))
heat_values = np.array(list(wifi_counts.values()))
# 無線LANが検出されない位置をフィルタリングする
mask = heat_values != 0
heat_positions = heat_positions[mask]
heat_values = heat_values[mask]

# The heatmap
visualize_heatmap(heat_positions, 
                  heat_values, 
                  floor_plan_filename, 
                  width_meter, 
                  height_meter, 
                  colorbar_title='count', 
                  title=f'WiFi Count',
                  g_size=755,
                  colorscale='temps')

## IBeacon (Bluetooth) 🔵🦷

> 📌**IBeacon**: ビーコンは、モバイルアプリケーションと連動して、ユーザーがビーコンから一定の距離に近づいたときにプッシュ通知を行うなど、ルールに基づいて特定のメッセージやアクションを起こします。

> ❗**Note**: 下の例の経路には多くのビーコンが設置されていますが、iBeaconが全く設置されていない部分も多くあります。この部分の精度は低くなる可能性があります。

#### 📐dBm (decibel milliwatts) - 絶対的なパワーの指標として使用され、数値が0に近いほど、信号強度が優れていることを意味する📐

In [ ]:
# The GitHub function
from main import extract_ibeacon_rssi

# Getting the iBeacon data
ibeacon_rssi = extract_ibeacon_rssi(mwi_datas)
print(f'This floor has {len(ibeacon_rssi.keys())} ibeacons.')
ibeacon_ummids = list(ibeacon_rssi.keys())
target_ibeacon = ibeacon_ummids[0]
heat_positions = np.array(list(ibeacon_rssi[target_ibeacon].keys()))
heat_values = np.array(list(ibeacon_rssi[target_ibeacon].values()))[:, 0]

# The heatmap
visualize_heatmap(heat_positions, 
                  heat_values, 
                  floor_plan_filename, 
                  width_meter, 
                  height_meter, 
                  colorbar_title='dBm', 
                  title='iBeacon RSSE',
                  g_size=755,
                  colorscale='temps')

In [ ]:
# ~ END of EXPERIMENT ~
wandb.finish()
# ~~~~~~~~~~~~~~~~~~~~~

# 3. Understanding the Competition Metric

> 📌**Note**: 多くのノートブックでは、同じ `comp_metric` 関数を使用して *Evaluation metric* を計算していますが（以下のように）、この変数とその意味を理解するのに少し時間がかかりました。そのため、（私のように）コンペの説明書に書かれている情報を早々に読み飛ばしてしまった人のために、少しだけ情報を共有することにしました。

`def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]`
    
### 説明

上記の式は、以下の数学関数のPython版に過ぎません（競技会で提供されたものです）:
<img src="https://i.imgur.com/1Z0s7Cc.png" width=500>

In [ ]:
from numpy import sqrt, power
from numpy import abs as absolute

In [ ]:
def mean_position_error(x_pred, y_pred, f_pred, x_true, y_true, f_true, p=15):
    '''Mean Position Errorを評価するカスタム関数。
    x: waypointの位置のx座標; dtype list()
    y: waypointの位置のy座標; dtype list()
    f: 正確な階数または建物; dtype list()
    p: フロアペナルティ'''
    
    N = len(x_true)
    #1
    formula = sqrt( power(x_pred - x_true, 2) + power(y_pred - y_true, 2) )
    #2
    formula = formula + p * absolute(f_pred - f_true)
    #3
    formula = formula.sum() / N
    
    return formula

### RAPIDS function
- RAPIDS：データサイエンスのための様々な処理を一貫してGPUで行うためのプラットフォームです。
- 参考：https://qiita.com/shin_ishiguro/items/8f39aac45acc8363a42e

In [ ]:
!pip install cupy-cuda102
!pip install cupy

In [ ]:
!pip freeze

In [ ]:
!pip install cupy --no-cache-dir -vvvv

In [ ]:
from cupy import sqrt as sqrt_g
from cupy import power as power_g
from cupy import abs as abs_g

In [ ]:
def mean_position_error_gpu(x_pred, y_pred, f_pred, x_true, y_true, f_true, p=15):
    '''Same, but Faster ;)
    ここでRAPIDSを使用することで、後にXGBoostモデルを使用することができます。'''
    
    N = len(x_true)
    formula = sqrt_g( power_g(x_pred - x_true, 2) + power_g(y_pred - y_true, 2) )
    formula = formula + p * abs_g(f_pred - f_true)
    formula = formula.sum() / N
    return formula

# 4. Baseline Model

> 📌**Note**: 前処理されたデータは[Devin Anzelmo](https://www.kaggle.com/devinanzelmo)による[this dataset](https://www.kaggle.com/devinanzelmo/indoor-navigation-and-location-wifi-features)からのものです。

In [ ]:
def make_submission(predictions, sample_subm, name="base.csv"):
    '''予測値のリストをデータフレーム形式で受け取ります。'''
    final_submission = pd.concat(predictions).reset_index(drop=True)
    final_submission.index = sample_subm.index
    final_submission.to_csv(name)
    print("Submission ready.")

## I. Light GBM
- 決定木アルゴリズムに基づいた勾配ブースティング（Gradient Boosting）の機械学習フレームワーク
- 参考：https://www.codexa.net/lightgbm-beginner/

In [ ]:
# Import Libraries
import lightgbm as lgb

# ~~~~
# Data
# ~~~~
base_dir = "../input/indoor-navigation-and-location-wifi-features/wifi_features"
train_dir = "/train/*_train.csv"
test_dir = "/test/*_test.csv"


# Paths for train & test files
train_paths = sorted(glob.glob(base_dir + train_dir))
test_paths = sorted(glob.glob(base_dir + test_dir))
sample_subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv',
                          index_col=0)

print("Len Train Files: {}".format(len(train_paths)), "\n" +
      "Len Test Files: {}".format(len(test_paths)))

In [ ]:
# 新しい実験の初期化 (LGBM)
run = wandb.init(project="indoor-location-kaggle", name="lgbm_train")

wandb.log({'Len Train Files' : len(train_paths),
           'Len Test Files' : len(test_paths)})

> Schema of the "Training loop":
<img src="https://i.imgur.com/UQmdRcz.png" width=750>

#### Code Below

In [ ]:
def train_lgbm(train_perc=0.75, version=1):
    '''
    Training loop
    '''

    f = open(f"lgbm_logs_{version}.txt", "w+")
    lgbm_predictions = []
    
    k = 1
    for train_path, test_path in zip(train_paths, test_paths):


        # --- Read in data ---
        # index_col=指定した列をインデックスとして利用
        # frac=抽出する行・列の割合を指定できる。1だと100%。
        train_df = pd.read_csv(train_path, index_col=0)
        train_df = train_df.sample(frac=1, random_state=10)

        # 最後の列を消す (which is "site_path_timestamp")
        test_df = pd.read_csv(test_path, index_col=0).iloc[:, :-1]

        # トレーニングとバリデーションデータのサンプルアウト
        ### 3つのモデルすべてに同じ情報を選択するように注意する必要があります。
        ### 1 for x, 1 for y and 1 for floor

        train_size = int(len(train_df) * train_perc)


        # --- Data Validation ---
        # Train features + targets(75％を練習用)
        X_train = train_df.iloc[:train_size, :-4]
        y_train_x = train_df.iloc[:train_size, -4]
        y_train_y = train_df.iloc[:train_size, -3]
        y_train_f = train_df.iloc[:train_size, -2]

        # Valid features + targets（２５％をテスト用）
        X_valid = train_df.iloc[train_size:, :-4]
        y_valid_x = train_df.iloc[train_size:, -4]
        y_valid_y = train_df.iloc[train_size:, -3]
        y_valid_f = train_df.iloc[train_size:, -2]


        # --- Model Training ---
        # n_estimators=ツリー（木）の数、num_leaves=最大ツリーリーフ数
        lgbm_x = lgb.LGBMRegressor(n_estimators=150, num_leaves=127)
        lgbm_x.fit(X_train, y_train_x)

        lgbm_y = lgb.LGBMRegressor(n_estimators=150, num_leaves=127)
        lgbm_y.fit(X_train, y_train_y)

        lgbm_f = lgb.LGBMClassifier(n_estimators=150, num_leaves=127)
        lgbm_f.fit(X_train, y_train_f)


        # --- モデル検証 予測 ---
        preds_x = lgbm_x.predict(X_valid)
        preds_y = lgbm_y.predict(X_valid)
        preds_f = lgbm_f.predict(X_valid).astype(int)
        
        mpe = mean_position_error(preds_x, preds_y, preds_f,
                                  y_valid_x, y_valid_y, y_valid_f)
        print("{} | MPE: {}".format(k, mpe))
        # Save logs
        with open(f"lgbm_logs_{version}.txt", 'a+') as f:
            print("{} | MPE: {}".format(k, mpe), file=f)
        k+=1
        
        # Log MPE of this experiment
        wandb.log({'MPE' : mpe}, step=k)


        # --- モデルテストの予測 ---
        test_preds_x = lgbm_x.predict(test_df)
        test_preds_y = lgbm_y.predict(test_df)
        test_preds_f = lgbm_f.predict(test_df).astype(int)

        all_test_preds = pd.DataFrame({'floor' : test_preds_f,
                                       'x' : test_preds_x, 
                                       'y' : test_preds_y})
        lgbm_predictions.append(all_test_preds)
    
    
    return lgbm_predictions

### Training

In [ ]:
# 独自のモデルを作成する場合は、以下の行をコメントアウトしてください。
lgbm_predictions = train_lgbm(train_perc = 0.75, version=1)

# Logs from my training:
print(open('../input/indoor-locationnavigation-2021/lgbm_logs_1.txt', "r").read())

> ❗**Attention**: *5番目*のデータフレームでは、大きなエラーが発生しました（平均4個から18個に増えました）。このケースは、モデルがアンダーフィッティングしていると思われるので、考慮しなければなりません。10番目、13番目、21番目のデータフレームも同様に大きなMPEを持っています。

### Submission LGBM

In [ ]:
# 独自のモデルを作成する場合は、以下の行をコメントアウトしてください。
make_submission(lgbm_predictions, sample_subm, name="lgbm_base.csv")

# My submission:
lgbm_predictions = pd.read_csv("../input/indoor-locationnavigation-2021/lgbm_base.csv")
lgbm_predictions.to_csv("lgbm_base.csv", index=False)

In [ ]:
# ~ END of EXPERIMENT ~
wandb.finish()
# ~~~~~~~~~~~~~~~~~~~~~

## II. XGBoost - Faster with RAPIDS

> 📌**Note**: 今回は、GPU上の**RAPIDS**ライブラリとXGBoostを組み合わせたものをベースモデルの一つとして使用します。このオープンソースのライブラリ群についての詳細は[こちら](https://rapids.ai/)をご覧ください。

In [ ]:
# Libraries
import cudf
import cupy
import cuml
import xgboost

# Adjust floor function
### Multiclass XGBoostは、[0, n]の間のラベルしか取らないので
### しかし、マイナスのフロア値を持っています
def adjust_floor(df, col_name):
    '''Adjusts the floor to be >= 0.
    Also returns the number fo classes (also used to complete classification).'''
    num_classes = df[col_name].nunique()
    smallest = df[col_name].unique().min()
    df[col_name] = df[col_name] - smallest
    
    return df[col_name], num_classes, smallest

In [ ]:
# Initialize new experiment (XGB)
run = wandb.init(project="indoor-location-kaggle", name="xgb_train")

wandb.log({'Len Train Files' : len(train_paths),
           'Len Test Files' : len(test_paths)})

In [ ]:
def train_xgb(train_perc=0.75, version=1):
    '''
    Training loop
    '''

    f = open(f"xgb_logs_{version}.txt", "w+")
    xgb_predictions = []
    
    k = 1
    for train_path, test_path in zip(train_paths, test_paths):


        # --- Read in data ---
        train_df = cudf.read_csv(train_path, index_col=0)
        train_df = train_df.sample(frac=1, random_state=10)
        train_df["f"], num_classes, smallest = adjust_floor(train_df, 'f')

        # Erase last column (which is "site_path_timestamp")
        test_df = cudf.read_csv(test_path, index_col=0).iloc[:, :-1]

        # Sample out training and validation data
        ### we need to be careful to choose same information for ALL 3 models
        ### 1 for x, 1 for y and 1 for floor

        train_size = int(len(train_df) * train_perc)


        # --- Data Validation ---
        # Train features + targets
        X_train = train_df.iloc[:train_size, :-4]
        y_train_x = train_df.iloc[:train_size, -4]
        y_train_y = train_df.iloc[:train_size, -3]
        y_train_f = train_df.iloc[:train_size, -2]

        # Valid features + targets
        X_valid = train_df.iloc[train_size:, :-4]
        y_valid_x = cupy.asanyarray(train_df.iloc[train_size:, -4])
        y_valid_y = cupy.asanyarray(train_df.iloc[train_size:, -3])
        y_valid_f = cupy.asanyarray(train_df.iloc[train_size:, -2])
        
        
        # --- Parameters ---
        # max_depth=決定木の深さの最大値  max_leaves=葉の最大値　tree_method=木構造アルゴリズム  grow_policy=ツリーに新しいノードを追加する方法
        regr_params = {'max_depth' : 4, 'max_leaves' : 2**4, 
                       'tree_method' : 'gpu_hist', 'objective' : 'reg:squarederror',
                       'grow_policy' : 'lossguide', 'colsample_bynode': 0.8}
        classif_params = {'max_depth' : 4, 'max_leaves' : 2**4,
                          'tree_method' : 'gpu_hist', 'objective' : 'multi:softmax',
                          'num_class' : num_classes, 'grow_policy' : 'lossguide',
                          'colsample_bynode': 0.8, 'verbosity' : 0}


        # --- Model Training ---
        trainMatrix_x = xgboost.DMatrix(data=X_train, label=y_train_x)
        xgboost_x = xgboost.train(params=regr_params, dtrain=trainMatrix_x)

        trainMatrix_y = xgboost.DMatrix(data=X_train, label=y_train_y)
        xgboost_y = xgboost.train(params=regr_params, dtrain=trainMatrix_y)

        trainMatrix_f = xgboost.DMatrix(data=X_train, label=y_train_f)
        xgboost_f = xgboost.train(params=classif_params, dtrain=trainMatrix_f)


        # --- Model Validation Predictions ---
        preds_x = cupy.asanyarray(xgboost_x.predict(xgboost.DMatrix(X_valid)))
        preds_y = cupy.asanyarray(xgboost_y.predict(xgboost.DMatrix(X_valid)))
        preds_f = cupy.asanyarray(xgboost_f.predict(xgboost.DMatrix(X_valid)).astype(int))

        mpe = mean_position_error_gpu(preds_x, preds_y, preds_f,
                                      y_valid_x, y_valid_y, y_valid_f)
        print("{} | MPE: {}".format(k, mpe))
        # Save logs
        with open(f"xgb_logs_{version}.txt", 'a+') as f:
            print("{} | MPE: {}".format(k, mpe), file=f)
        k+=1
        # Log MPE of this experiment
        wandb.log({'MPE' : mpe}, step=k)


        # --- Model Test Predictions ---
        test_preds_x = xgboost_x.predict(xgboost.DMatrix(test_df))
        test_preds_y = xgboost_y.predict(xgboost.DMatrix(test_df))
        test_preds_f = xgboost_f.predict(xgboost.DMatrix(test_df)).astype(int) + smallest

        all_test_preds = pd.DataFrame({'floor' : test_preds_f,
                                       'x' : test_preds_x, 
                                       'y' : test_preds_y})
        xgb_predictions.append(all_test_preds)
    
    
    return xgb_predictions

### Training

In [ ]:
# 独自のモデルを作成する場合は、以下の行をコメントを外してください。
# xgb_predictions = train_xgb(train_perc=0.75, version=1)

# Logs from my training:
print(open('../input/indoor-locationnavigation-2021/xgb_logs_1.txt', "r").read())

### Submission

In [ ]:
# 下の行のコメントを外して、自分で投稿してください。
# make_submission(xgb_predictions, sample_subm, name="xgb_base.csv")

# My submission:
xgb_predictions = pd.read_csv("../input/indoor-locationnavigation-2021/xgb_base.csv")
xgb_predictions.to_csv("xgb_base.csv", index=False)

In [ ]:
# ~ END of EXPERIMENT ~
wandb.finish()
# ~~~~~~~~~~~~~~~~~~~~~

# 5. Save W&B Submissions and Logs

> We can save the predictions and logs to W&B.

In [ ]:
# Submissions
### we can save the predictions in W&B
run = wandb.init(project='indoor-location-kaggle', name='submissions')
artifact = wandb.Artifact(name='submissions', 
                          type='dataset')

artifact.add_file("../input/indoor-locationnavigation-2021/lgbm_base.csv")
artifact.add_file("../input/indoor-locationnavigation-2021/xgb_base.csv")

wandb.log_artifact(artifact)
wandb.finish()

In [ ]:
# Logs
### we can save the predictions in W&B
run = wandb.init(project='indoor-location-kaggle', name='training_logs')
artifact = wandb.Artifact(name='training_logs', 
                          type='dataset')

artifact.add_file("../input/indoor-locationnavigation-2021/lgbm_logs_1.txt")
artifact.add_file("../input/indoor-locationnavigation-2021/xgb_logs_1.txt")

wandb.log_artifact(artifact)
wandb.finish()

# モデルの組み合わせ

In [ ]:
# Read in data
lgb_preds = pd.read_csv("../input/indoor-locationnavigation-2021/lgbm_base.csv")
xgb_preds = pd.read_csv("../input/indoor-locationnavigation-2021/xgb_base.csv")

sample_submission = pd.read_csv("../input/indoor-location-navigation/sample_submission.csv")

# Sample Submission
sample_submission["x"] = lgb_preds["x"] * 0.9 + xgb_preds["x"] * 0.1
sample_submission["y"] = lgb_preds["y"] * 0.9 + xgb_preds["y"] * 0.1

sample_submission.to_csv("blend1.csv", index=False)

<img src="https://i.imgur.com/cUQXtS7.png">

# Specs on how I prepped & trained ⌨️🎨
### (on my local machine)
* Z8 G4 Workstation 🖥
* 2 CPUs & 96GB Memory 💾
* NVIDIA Quadro RTX 8000 🎮
* RAPIDS version 0.17 🏃🏾‍♀️